In [25]:
url = 'https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=14450;type=tournament'

In [26]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import re

In [27]:
# To Store All The Match Links
match_links = []
# To Store All The Player Links
player_links = []

In [28]:
driver = webdriver.Chrome()

driver.get(url)

wait = WebDriverWait(driver, 10)

rows = driver.find_elements(By.TAG_NAME, 'tr')

with open('dataset/match_summary.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    for row in rows:
        cols = row.find_elements(By.TAG_NAME, 'td')
        if cols:
            data = [col.text for col in cols]
            if cols[6].find_elements(By.XPATH, ".//a"):
                match_links.append([cols[0].text,cols[1].text,cols[6].text,cols[6].find_element(By.XPATH, ".//a").get_attribute("href")])
            writer.writerow(data)

driver.quit()

In [30]:
batting = [["match", "teamInnings", "battingPos", "batsmanName", "runs", "balls", "4s",	"6s", "SR",	"out/not_out", "match_id"]]
bowling = [["match",	"bowlingTeam",	"bowlerName",	"overs",	"maiden",	"runs",	"wickets",	"economy",	"0s",	"4s",	"6s",	"wides",	"noBalls",	"match_id"]]

In [31]:
for data in match_links:
    driver = webdriver.Chrome()

    driver.get(data[3])

    wait = WebDriverWait(driver, 10)

    rows = driver.find_elements(By.XPATH, ".//div[contains(concat(' ', normalize-space(@class), ' '), ' ds-rounded-lg ') and contains(concat(' ', normalize-space(@class), ' '), ' ds-mt-2 ')]")

    for row in rows:
        active_team = row.find_element(By.XPATH, ".//span[contains(concat(' ', normalize-space(@class), ' '), ' ds-text-title-xs ') and contains(concat(' ', normalize-space(@class), ' '), ' ds-font-bold ') and contains(concat(' ', normalize-space(@class), ' '), ' ds-capitalize ')]").text
        tables = row.find_elements(By.TAG_NAME, 'table')
        for j, table in enumerate(tables):
            if j < 1:
                flag = 1
                bat_pos = 0
                for i,tr in enumerate(table.find_elements(By.TAG_NAME, 'tr')):
                    if tr.text != "": 
                        if (i > 0):
                            if (flag):
                                if "ds-text-tight-s" in tr.get_attribute("class"):
                                    flag = 0
                                else:
                                    temp = []
                                    bat_pos += 1
                                    for k,td in enumerate(tr.find_elements(By.TAG_NAME, 'td')):
                                        if td.find_elements(By.XPATH, ".//a"):
                                            player_links.append(td.find_element(By.XPATH, ".//a").get_attribute("href"))
                                        if k == 0:
                                            temp.append(re.sub(r'\s*\(.*?\)\s*|[†*]', '', td.text))
                                        elif k == 1:
                                            temp.append("not_out") if td.text == "not out" else temp.append("out")
                                        else:
                                            temp.append(td.text)
                                    batting.append([data[0] + " vs " + data[1],active_team,bat_pos,temp[0],temp[2],temp[3],temp[5],temp[6],temp[7],temp[1],data[2]])  
                flag = 1
            else:
                for i,tr in enumerate(table.find_elements(By.TAG_NAME, 'tr')):
                    if tr.text != "": 
                        if (i > 0):
                            temp = []
                            for k,td in enumerate(tr.find_elements(By.TAG_NAME, 'td')):
                                if td.find_elements(By.XPATH, ".//a"):
                                    player_links.append(td.find_element(By.XPATH, ".//a").get_attribute("href"))
                                if k == 0:
                                    temp.append(re.sub(r'\s*\(.*?\)\s*|[†*]', '', td.text))
                                else:
                                    temp.append(td.text)
                            if active_team == data[0]: temp_active = data[1] 
                            elif active_team == data[1]: temp_active = data[0]    
                            bowling.append([data[0] + " vs " + data[1],temp_active,temp[0],temp[1],temp[2],temp[3],temp[4],temp[5],temp[6],temp[7],temp[8],temp[9],temp[10],data[2]])
    driver.quit()

In [32]:
with open('dataset/match_batting_summary.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(batting)

with open('dataset/match_bowling_summary.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(bowling)


In [33]:
player_links = list(set(player_links))

In [42]:
with open('dataset/players_links.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(player_links)

In [47]:
player_data = [["name","team","image","battingStyle","bowlingStyle","playingRole","description"]]

In [58]:
for link in  player_links:

    driver = webdriver.Chrome()

    driver.get(link)

    wait = WebDriverWait(driver, 10)

    name = ""
    country = ""
    image = ""
    battingStyle = ""
    bowlingStyle = ""
    playingRole = ""
    description = ""

    name = driver.find_element(By.XPATH, ".//h1[contains(concat(' ', normalize-space(@class), ' '), ' ds-text-title-l ') and contains(concat(' ', normalize-space(@class), ' '), ' ds-font-bold ')]").text

    country = driver.find_element(By.XPATH, ".//span[contains(concat(' ', normalize-space(@class), ' '), ' ds-text-comfortable-s ')]").text

    if driver.find_elements(By.XPATH, ".//div[contains(concat(' ', normalize-space(@class), ' '), ' ci-player-bio-content ') ]"):
        for des in driver.find_elements(By.XPATH, ".//div[contains(concat(' ', normalize-space(@class), ' '), ' ci-player-bio-content ') ]"):
            description = des.find_element(By.TAG_NAME, 'p').text

    p = driver.find_elements(By.XPATH, ".//div[contains(concat(' ', normalize-space(@class), ' '), ' ds-grid ') and contains(concat(' ', normalize-space(@class), ' '), ' lg:ds-grid-cols-3 ') and contains(concat(' ', normalize-space(@class), ' '), ' ds-grid-cols-2 ') and contains(concat(' ', normalize-space(@class), ' '), ' ds-gap-4 ') and contains(concat(' ', normalize-space(@class), ' '), ' ds-mb-8 ')]")

    for ps in p:
        cols = ps.find_elements(By.TAG_NAME, 'div')
        for col in cols:
            if col.find_elements(By.XPATH, ".//p[contains(concat(' ', normalize-space(@class), ' '), ' ds-text-tight-m ') and contains(concat(' ', normalize-space(@class), ' '), ' ds-font-regular ') and contains(concat(' ', normalize-space(@class), ' '), ' ds-uppercase ') and contains(concat(' ', normalize-space(@class), ' '), ' ds-text-typo-mid3 ')]"):
                text = col.find_element(By.XPATH, ".//p[contains(concat(' ', normalize-space(@class), ' '), ' ds-text-tight-m ') and contains(concat(' ', normalize-space(@class), ' '), ' ds-font-regular ') and contains(concat(' ', normalize-space(@class), ' '), ' ds-uppercase ') and contains(concat(' ', normalize-space(@class), ' '), ' ds-text-typo-mid3 ')]").text
                content = col.find_element(By.XPATH, ".//span[contains(concat(' ', normalize-space(@class), ' '), ' ds-text-title-s ') and contains(concat(' ', normalize-space(@class), ' '), ' ds-font-bold ') and contains(concat(' ', normalize-space(@class), ' '), ' ds-text-typo ')]").text
                if (text == "BATTING STYLE"):
                    battingStyle = content
                elif (text == "BOWLING STYLE"):
                    bowlingStyle = content
                elif (text == "PLAYING ROLE"):
                    playingRole = content


    if driver.find_elements(By.XPATH, ".//div[contains(concat(' ', normalize-space(@class), ' '), ' ds-ml-auto ') and contains(concat(' ', normalize-space(@class), ' '), ' ds-w-48 ') and contains(concat(' ', normalize-space(@class), ' '), ' ds-h-48 ')]"):
        for img in driver.find_elements(By.XPATH, ".//div[contains(concat(' ', normalize-space(@class), ' '), ' ds-ml-auto ') and contains(concat(' ', normalize-space(@class), ' '), ' ds-w-48 ') and contains(concat(' ', normalize-space(@class), ' '), ' ds-h-48 ')]"):
            tem = tem = "//img[@alt and contains(concat(' ', normalize-space(@alt), ' '), '{}')]".format(name)
            image = img.find_element(By.XPATH, tem).get_attribute("src")


    player_data.append([name, country, image, battingStyle, bowlingStyle, playingRole, description])

    driver.quit()

In [61]:
with open('dataset/players.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(player_data)